In [128]:
import numpy as np 
import matplotlib.pyplot as plt

# Used for Confusion Matrix
from sklearn import metrics
import seaborn as sns

# Used for Loading MNIST
from struct import unpack

%matplotlib inline

In [129]:
from sklearn import datasets
import random
from random import randrange
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 

In [130]:
X = datasets.load_digits().data
y = datasets.load_digits().target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [131]:
def create_model(model_type):
    if model_type == 1:
        model = LogisticRegression(solver="lbfgs", max_iter=10000)
    return model

num_devices = 10
model_type = 1
num_epochs = 2


def main(model_type):
    X_train_split = np.array_split(X_train, num_devices)
    y_train_split = np.array_split(y_train, num_devices)
    test_model = create_model(model_type)
    test_model.fit(X_train[0:50], y_train[0:50])
    avg_coef = np.zeros(np.shape(test_model.coef_))
    avg_intercept = np.zeros(np.shape(test_model.intercept_))
    for epoch in range(num_epochs):
        for k in range(num_devices):
            model = create_model(model_type)
            if epoch < 1:
                model.fit(X_train_split[k], y_train_split[k])
            else:
                model.fit(X_train[0:50], y_train[0:50])
                model.coef_ = avg_coef/(num_devices*epoch)
                model.intercept_ = avg_intercept/(num_devices*epoch)
                model.fit(X_train_split[k], y_train_split[k])
            avg_coef += model.coef_
            avg_intercept += model.intercept_
    
    final_coef = avg_coef/(num_devices*num_epochs)
    final_intercept = avg_intercept/(num_devices*num_epochs)
    final_model = create_model(model_type)
    final_model.fit(X_train[0:50], y_train[0:50])
    final_model.coef_ = final_coef
    final_model.intercept_ = final_intercept
    
    return final_model.score(X_test, y_test)
    


In [127]:
score = main(1)
print(score)

0.9494949494949495
